In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline  

import numpy as np

In [7]:
from fnsa.entity import ENScopeDetector
from fnsa.lexicon import get_en2scope, Lexicon
from fnsa.scope import DRScopeDetector, IFScopeDetector
from fnsa.graph import make_graph
from fnsa.util import *
import spacy
nlp = spacy.load('en_core_web_sm')
sentences = load_sentences()

In [8]:
lexicon = Lexicon(nlp)
dr_detector = DRScopeDetector()
if_detector = IFScopeDetector()
en_detector = ENScopeDetector()

In [9]:
def show_entity_scope(sentence, detectors=[dr_detector, en_detector, if_detector]):
    print(sentence)
    doc = lexicon(sentence)
    graph = make_graph(doc)
    lex2tokens = build_lex2tokens(doc)
    for detector in detectors: detector(doc, graph, lex2tokens)
    en2scope = get_en2scope(doc)
    for index, ent in enumerate(doc.ents): 
        if ent[0].i in en2scope:
            tokens = [token for token in doc if token.i in en2scope[ent[0].i]]
            tokens = sorted(tokens, key=lambda token: token.i)
            print("%d. %s -->> %s" % (index + 1, ent.text, " ".join(annotate_token(token) for token in tokens)))
    return doc

In [10]:
for sentence in sentences: 
    doc = show_entity_scope(sentence, detectors=[dr_detector, en_detector, if_detector])
    print("\n")

General Electric reports its operating profit increased to $20.5 million.
1. General Electric -->> [General Electric/en:ORG] reports its [operating profit/fe:+] [increased/dr:+] to [$20.5 million/en:MONEY] .


Ford Motor Company reported higher full-year profits on Thursday boosted by a buoyant period for its car loans unit.
1. Ford Motor Company -->> [Ford Motor Company/en:ORG] reported [higher/dr:+] [full-year profits/fe:+] on [Thursday/en:DATE] [boosted/dr:+] by a [buoyant period/fi:+] for its car loans unit .


By agreeing to stay the proceedings, AT&T avoided a potential bombshell.
1. AT&T -->> By agreeing to stay the proceedings , [AT&T/en:ORG] [avoided/if:rev] a [potential bombshell/fi:-] .


Shares of Standard Chartered rose 1.2%, while Royal Bank of Scotland shares rose 2% and Barclays shares were down 1.7%.
1. Standard Chartered -->> [Shares/fe:+] of [Standard Chartered/en:ORG] [rose/dr:+] [1.2%/en:PERCENT] , [rose/dr:+] .
3. Royal Bank -->> [Shares/fe:+] [rose/dr:+] [1.2%/en